# Υλοποίηση ενός Συστήματος Ειδικού για Ζώα

Ένα παράδειγμα από το [AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners).

Σε αυτό το παράδειγμα, θα υλοποιήσουμε ένα απλό σύστημα βασισμένο στη γνώση για να προσδιορίσουμε ένα ζώο με βάση ορισμένα φυσικά χαρακτηριστικά. Το σύστημα μπορεί να αναπαρασταθεί από το παρακάτω δέντρο AND-OR (αυτό είναι ένα μέρος του συνολικού δέντρου, μπορούμε εύκολα να προσθέσουμε περισσότερους κανόνες):

![](../../../../translated_images/AND-OR-Tree.5592d2c70187f283703c8e9c0d69d6a786eb370f4ace67f9a7aae5ada3d260b0.el.png)


## Το δικό μας κέλυφος συστημάτων ειδικών γνώσεων με οπισθοδρομική συμπερασματολογία

Ας προσπαθήσουμε να ορίσουμε μια απλή γλώσσα για την αναπαράσταση γνώσης βασισμένη σε κανόνες παραγωγής. Θα χρησιμοποιήσουμε κλάσεις της Python ως λέξεις-κλειδιά για τον ορισμό κανόνων. Υπάρχουν ουσιαστικά 3 τύποι κλάσεων:
* `Ask` αντιπροσωπεύει μια ερώτηση που πρέπει να γίνει στον χρήστη. Περιέχει το σύνολο των πιθανών απαντήσεων.
* `If` αντιπροσωπεύει έναν κανόνα και είναι απλώς συντακτική διευκόλυνση για την αποθήκευση του περιεχομένου του κανόνα.
* `AND`/`OR` είναι κλάσεις που αντιπροσωπεύουν τα κλαδιά AND/OR του δέντρου. Απλώς αποθηκεύουν τη λίστα των επιχειρημάτων μέσα. Για την απλοποίηση του κώδικα, όλη η λειτουργικότητα ορίζεται στην γονική κλάση `Content`.


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

Στο σύστημά μας, η ενεργή μνήμη θα περιέχει τη λίστα των **γεγονότων** ως **ζεύγη χαρακτηριστικών-τιμών**. Η βάση γνώσεων μπορεί να οριστεί ως ένα μεγάλο λεξικό που αντιστοιχεί ενέργειες (νέα γεγονότα που πρέπει να εισαχθούν στην ενεργή μνήμη) σε συνθήκες, εκφρασμένες ως εκφράσεις AND-OR. Επίσης, ορισμένα γεγονότα μπορούν να `ζητηθούν`.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

Για να πραγματοποιήσουμε την αντίστροφη συμπερασματολογία, θα ορίσουμε την κλάση `Knowledgebase`. Αυτή θα περιλαμβάνει:
* Λειτουργική `μνήμη` - ένα λεξικό που αντιστοιχεί χαρακτηριστικά σε τιμές
* `Κανόνες` της βάσης γνώσης στη μορφή που ορίστηκε παραπάνω

Δύο βασικές μέθοδοι είναι:
* `get` για την απόκτηση της τιμής ενός χαρακτηριστικού, πραγματοποιώντας συμπερασματολογία αν είναι απαραίτητο. Για παράδειγμα, `get('color')` θα πάρει την τιμή μιας θέσης χρώματος (θα ρωτήσει αν είναι απαραίτητο και θα αποθηκεύσει την τιμή για μελλοντική χρήση στη λειτουργική μνήμη). Αν ρωτήσουμε `get('color:blue')`, θα ζητήσει ένα χρώμα και στη συνέχεια θα επιστρέψει τιμή `y`/`n` ανάλογα με το χρώμα.
* `eval` πραγματοποιεί την πραγματική συμπερασματολογία, δηλαδή διατρέχει το δέντρο AND/OR, αξιολογεί υπο-στόχους, κλπ.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Τώρα ας ορίσουμε τη βάση γνώσεων για τα ζώα και να πραγματοποιήσουμε τη συμβουλευτική. Σημειώστε ότι αυτή η κλήση θα σας κάνει ερωτήσεις. Μπορείτε να απαντήσετε πληκτρολογώντας `y`/`n` για ερωτήσεις ναι-όχι, ή καθορίζοντας αριθμό (0..N) για ερωτήσεις με περισσότερες επιλογές απαντήσεων.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Χρήση του PyKnow για Προωθητική Συμπερασματολογία

Στο επόμενο παράδειγμα, θα προσπαθήσουμε να υλοποιήσουμε προωθητική συμπερασματολογία χρησιμοποιώντας μία από τις βιβλιοθήκες για αναπαράσταση γνώσης, [PyKnow](https://github.com/buguroo/pyknow/). Το **PyKnow** είναι μια βιβλιοθήκη για τη δημιουργία συστημάτων προωθητικής συμπερασματολογίας σε Python, η οποία έχει σχεδιαστεί ώστε να μοιάζει με το κλασικό παλιό σύστημα [CLIPS](http://www.clipsrules.net/index.html).

Θα μπορούσαμε επίσης να υλοποιήσουμε την προωθητική αλυσίδωση μόνοι μας χωρίς ιδιαίτερα προβλήματα, αλλά οι απλοϊκές υλοποιήσεις συνήθως δεν είναι πολύ αποδοτικές. Για πιο αποτελεσματική αντιστοίχιση κανόνων χρησιμοποιείται ένας ειδικός αλγόριθμος, [Rete](https://en.wikipedia.org/wiki/Rete_algorithm).


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/buguroo/pyknow/

  Cloning https://github.com/buguroo/pyknow/ to /tmp/pip-req-build-3cqeulyl
  Running command git clone --filter=blob:none --quiet https://github.com/buguroo/pyknow/ /tmp/pip-req-build-3cqeulyl
  Resolved https://github.com/buguroo/pyknow/ to commit 48818336f2e9a126f1964f2d8dc22d37ff800fe8
  Preparing metadata (setup.py) ... done
  Using cached frozendict-1.2.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... done
  Using cached schema-0.6.7-py2.py3-none-any.whl (14 kB)
  Created wheel for pyknow: filename=pyknow-1.7.0-py3-none-any.whl size=34228 sha256=b7de5b09292c4007667c72f69b98d5a1b5f7324ff15f9dd8e077c3d5f7aade42
  Stored in directory: /tmp/pip-ephem-wheel-cache-k7jpave7/wheels/81/1a/d3/f6c15dbe1955598a37755215f2a10449e7418500d7bd4b9508
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3148 sha256=2863d55c240d2409cddf05ccfe600591f8478681549fc97555c47c90dc6bb160
  Stored in directory: /home/rg/.cache/pip/wheels/49/ac/f8/cb8120244e710bdb479c86198b03c7b08c

In [13]:
from pyknow import *
#import pyknow

Θα ορίσουμε το σύστημά μας ως μια κλάση που υποκλάση την `KnowledgeEngine`. Κάθε κανόνας ορίζεται από μια ξεχωριστή συνάρτηση με την αναnotation `@Rule`, η οποία καθορίζει πότε πρέπει να ενεργοποιηθεί ο κανόνας. Μέσα στον κανόνα, μπορούμε να προσθέσουμε νέα δεδομένα χρησιμοποιώντας τη συνάρτηση `declare`, και η προσθήκη αυτών των δεδομένων θα έχει ως αποτέλεσμα να κληθούν κάποιοι επιπλέον κανόνες από τη μηχανή προώθησης συμπερασμάτων.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Αφού ορίσουμε μια βάση γνώσεων, γεμίζουμε τη λειτουργική μας μνήμη με κάποια αρχικά γεγονότα και στη συνέχεια καλούμε τη μέθοδο `run()` για να εκτελέσουμε την εξαγωγή συμπερασμάτων. Μπορείτε να δείτε ως αποτέλεσμα ότι νέα συμπεράσματα προστίθενται στη λειτουργική μνήμη, συμπεριλαμβανομένου του τελικού γεγονότος σχετικά με το ζώο (αν έχουμε ορίσει σωστά όλα τα αρχικά γεγονότα).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])


---

**Αποποίηση ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης AI [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε κάθε προσπάθεια για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτόματες μεταφράσεις ενδέχεται να περιέχουν σφάλματα ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
